# Generate special quasi-random structures
*2025-03-07*

Demonstrates how to generate special quasi-random structures (SQSs) to study HEOs (here specifically, rock salt oxides).
- only show clusters including 1st and 2nd near-neighbor cations for clarity

A nice writeup for SQSs can be found at https://icet.materialsmodeling.org/advanced_topics/sqs_generation.html.

The original paper from Alex Zunger on SQSs can be found at https://doi.org/10.1103/PhysRevLett.65.353.

# Specific examples ->

- 4x4x4 SQS of 2-cation rock salt $Mg_{1/2}Ni_{1/2}O$.
- 4x4x4 SQS of 5-cation rock salt HEO $Mg_{1/5}Co_{1/5}Ni_{1/5}Cu_{1/5}Zn_{1/5}O$.
- 10x10x10 SQS of 5-cation rock salt HEO $Mg_{1/5}Co_{1/5}Ni_{1/5}Cu_{1/5}Zn_{1/5}O$.

In [1]:
# import necessary pytheos module for generating SQS and writing structure files
from pytheos.structure import generation, utils

In [3]:
# make a rocksalt MgO unit cell using ASE

from ase.build import bulk
from ase.visualize import view

unitcell = bulk("MgO", "rocksalt", a=4.21, cubic=False)
print(unitcell)

Atoms(symbols='MgO', pbc=True, cell=[[0.0, 2.105, 2.105], [2.105, 0.0, 2.105], [2.105, 2.105, 0.0]])


## $Mg_{1/2}Ni_{1/2}O$ (4x4x4 primitive supercell)

In [4]:
# generate SQS
sqs = generation.make_sqs(
    struc = unitcell,
    dimensions = (4, 4, 4),
    chemical_symbols = [ # this needs to match the cation-anion ordering of unit cell
        ["Mg", "Ni"],
        ["O"],
    ],
    cutoffs = [5.0], # some reasonable pair cutoff - 2nd NN here
    concentrations = { # don't need to include oxygen since only one species on the anion sublattice
        "Mg": 1/2,
        "Ni": 1/2,
    },
    num_steps = 10000, # not too many monte-carlo steps are needed for chemically-simpler compositions
)

icet.orbit_list: INFO  Done getting matrix_of_equivalent_positions.
icet.orbit_list: INFO  Done getting neighbor lists.
icet.orbit_list: INFO  Transformation of matrix of equivalent positions to lattice neighbor format completed.
icet.orbit_list: INFO  Finished construction of orbit list.
====================================== Cluster Space ======================================
 space group                            : Fm-3m (225)
 chemical species                       : ['Mg', 'Ni'] (sublattice A)
 cutoffs                                : 5.0000
 total number of parameters             : 4
 number of parameters by order          : 0= 1  1= 1  2= 2
 fractional_position_tolerance          : 2e-06
 position_tolerance                     : 1e-05
 symprec                                : 1e-05
-------------------------------------------------------------------------------------------
index | order |  radius  | multiplicity | orbit_index | multicomponent_vector | sublattices
--------------

In [ ]:
utils.write_from_ase_atoms(struc=sqs, file_path="./docs/sqs_Mg32Ni32O64.vasp")

<img src="./docs/sqs_Mg32Ni32O64.png" width="500">

Notice that for this $Mg_{1/2}Ni_{1/2}O$ composition that we obtain an ICET "best score" of -2.105 Angstroms, which is the furthest radius in our cluster space. This indicates that given the cluster space we have specified that this structure matches the "fully disordered limit". We also observed this by comparing the outputted "Trial Cluster Vector" and "Perfectly Random Cluster Vectors" above, where we see that these two vectors match exactly.

We will see for the 5-cation SQS that this number of MC steps does not achieve the same level of randomness due to the increased number of cation species.

## $Mg_{1/5}Co_{1/5}Ni_{1/5}Cu_{1/5}Zn_{1/5}O$ (4x4x4 primitive supercell)

In [7]:
# generate SQS
sqs = generation.make_sqs(
    struc = unitcell,
    dimensions = (4, 4, 4),
    chemical_symbols = [ # this needs to match the cation-anion ordering of unit cell
        ["Mg", "Co", "Ni", "Cu", "Zn"],
        ["O"],
    ],
    cutoffs = [5.0], # some reasonable pair cutoff - 2nd NN here
    concentrations = { # don't need to include oxygen since only one species on the anion sublattice
        "Mg": 13/64,
        "Co": 13/64,
        "Ni": 13/64,
        "Cu": 13/64,
        "Zn": 12/64, # slightly off-equimolar due to the supercell constraints
    },
    num_steps = 10000, # usually want more than this, but just for demonstration purposes here. Oftentimes values in the millions are desirable
)

icet.orbit_list: INFO  Done getting matrix_of_equivalent_positions.
icet.orbit_list: INFO  Done getting neighbor lists.
icet.orbit_list: INFO  Transformation of matrix of equivalent positions to lattice neighbor format completed.
icet.orbit_list: INFO  Finished construction of orbit list.
====================================== Cluster Space ======================================
 space group                            : Fm-3m (225)
 chemical species                       : ['Co', 'Cu', 'Mg', 'Ni', 'Zn'] (sublattice A)
 cutoffs                                : 5.0000
 total number of parameters             : 25
 number of parameters by order          : 0= 1  1= 4  2= 20
 fractional_position_tolerance          : 2e-06
 position_tolerance                     : 1e-05
 symprec                                : 1e-05
-------------------------------------------------------------------------------------------
index | order |  radius  | multiplicity | orbit_index | multicomponent_vector | sublat

In [ ]:
utils.write_from_ase_atoms(struc=sqs, file_path="./docs/sqs_Mg13Co13Ni13Cu13Zn12O64.vasp")

<img src="./docs/sqs_Mg13Co13Ni13Cu13Zn12O64.png" width="500">

Notice how much higher the "best value" is for our 5-cation HEO compared to the 2-cation composition. This is also reflected in the cluster vectors. One should usually use at least one-million MC steps for chemically complex SQS generation (*the more the better almost always*). Due to size constraints imposed by the heavy computational cost of DFT, you cannot always achieve a "perfectly random" structure for many-cation systems, therefore you must find a good balance between computational cost of the following DFT calculation and the size/disorder of your supercell...

##  $Mg_{1/5}Co_{1/5}Ni_{1/5}Cu_{1/5}Zn_{1/5}O$ (10x10x10 primitive supercell)
Example of a many-cation HEO that is *far* too expensive for DFT calculations, however could be run with a machine-learning interatomic potential if desired... This is to demonstrate that extremely large supercells are needed to achieve "perfectly random" SQSs for HEOs (we don't even achieve it with this 2000-atom SQS!).

*FYI we likely also get "better" cluster vectors as this is now equimolar cation concentrations since we are not limited to a size DFT calculations can be run*

In [9]:
# generate SQS
sqs = generation.make_sqs(
    struc = unitcell,
    dimensions = (10, 10, 10),
    chemical_symbols = [ # this needs to match the cation-anion ordering of unit cell
        ["Mg", "Co", "Ni", "Cu", "Zn"],
        ["O"],
    ],
    cutoffs = [5.0], # some reasonable pair cutoff - 2nd NN here
    concentrations = { # don't need to include oxygen since only one species on the anion sublattice
        "Mg": 1/5,
        "Co": 1/5,
        "Ni": 1/5,
        "Cu": 1/5,
        "Zn": 1/5,
    },
    num_steps = 10000, # usually want more than this, but just for demonstration purposes here. Oftentimes values in the millions are desirable
)

icet.orbit_list: INFO  Done getting matrix_of_equivalent_positions.
icet.orbit_list: INFO  Done getting neighbor lists.
icet.orbit_list: INFO  Transformation of matrix of equivalent positions to lattice neighbor format completed.
icet.orbit_list: INFO  Finished construction of orbit list.
====================================== Cluster Space ======================================
 space group                            : Fm-3m (225)
 chemical species                       : ['Co', 'Cu', 'Mg', 'Ni', 'Zn'] (sublattice A)
 cutoffs                                : 5.0000
 total number of parameters             : 25
 number of parameters by order          : 0= 1  1= 4  2= 20
 fractional_position_tolerance          : 2e-06
 position_tolerance                     : 1e-05
 symprec                                : 1e-05
-------------------------------------------------------------------------------------------
index | order |  radius  | multiplicity | orbit_index | multicomponent_vector | sublat

In [ ]:
utils.write_from_ase_atoms(struc=sqs, file_path="./docs/sqs_Mg200Co200Ni200Cu200Zn200O1000.vasp")

<img src="./docs/sqs_Mg200Co200Ni200Cu200Zn200O1000.png" width="500">